Cleaning Data in SQL

In [ ]:
SELECT *
FROM [Portfolio].[dbo].[nashville_housing]

---------------------------------------------------------------------------------------------
-- Populate Property Address Data

SELECT *
FROM nashville_housing
-- WHERE PropertyAddress is null 
ORDER BY ParcelID


SELECT a.ParcelID, a.PropertyAddress, b.ParcelID, b.PropertyAddress, ISNULL(A.PropertyAddress, b.PropertyAddress)
FROM nashville_housing a 
JOIN nashville_housing b 
    on a.ParcelID = b.ParcelID
    AND a.[UniqueID] <> b.[UniqueID]
WHERE a.PropertyAddress is null


UPDATE a 
SET PropertyAddress = ISNULL(A.PropertyAddress, b.PropertyAddress)
FROM nashville_housing a
JOIN nashville_housing b 
    on a.ParcelID = b.ParcelID
    AND a.[UniqueID] <> b.[UniqueID]
WHERE a.PropertyAddress is null


-----------------------------------------------

-- Breaking out Property and Owner Address into Individual Columns (Address, City, State)

Select PropertyAddress
FROM nashville_housing

SELECT 
SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress) -1) AS Address
, SUBSTRING(PropertyAddress, CHARINDEX(',', PropertyAddress) +1 , LEN(PropertyAddress)) AS Address
FROM nashville_housing

ALTER TABLE nashville_housing
Add PropertySplitAddress NVARCHAR(255);

Update nashville_housing
SET PropertySplitAddress = SUBSTRING(PropertyAddress, 1, CHARINDEX(',', PropertyAddress) -1)

ALTER TABLE nashville_housing
Add PropertySplitCity NVARCHAR(255);

Update nashville_housing
SET PropertySplitCity = SUBSTRING(PropertyAddress, CHARINDEX(',', PropertyAddress) +1 , LEN(PropertyAddress))

------------------------------------------------------------------------------

Select OwnerAddress
From nashville_housing

Select 
PARSENAME(REPLACE(OwnerAddress, ',', '.'), 3)
, PARSENAME(REPLACE(OwnerAddress, ',', '.'), 2)
, PARSENAME(REPLACE(OwnerAddress, ',', '.'), 1)
From nashville_housing



ALTER TABLE nashville_housing
Add OwnerSplitAdress NVARCHAR(255);

Update nashville_housing
SET OwnerSplitAdress = PARSENAME(REPLACE(OwnerAddress, ',', '.'), 3)

ALTER TABLE nashville_housing
Add OwnerSplitCity NVARCHAR(255);

Update nashville_housing
SET OwnerSplitCity = PARSENAME(REPLACE(OwnerAddress, ',', '.'), 2)

ALTER TABLE nashville_housing
Add OwnerSplitState NVARCHAR(255);

Update nashville_housing
SET OwnerSplitState = PARSENAME(REPLACE(OwnerAddress, ',', '.'), 1)


-----------------------------------------------------------------

-- Change Y and N to Yes and No in "Sold as Vacant" field 

Select Distinct(SoldAsVacant), Count(SoldAsVacant)
FROM nashville_housing
Group by SoldAsVacant
Order by 2

Select SoldAsVacant
, CASE When SoldAsVacant = 'Y' THEN 'Yes'
       When SoldAsVacant = 'N' THEN 'No'
       ELSE SoldAsVacant
       END
FROM nashville_housing

Update nashville_housing
SET SoldAsVacant = CASE When SoldAsVacant = 'Y' THEN 'Yes'
       When SoldAsVacant = 'N' THEN 'No'
       ELSE SoldAsVacant
       END

--------------------------------------------------------------------------

-- Remove Duplicate

WITH RowNumCTE AS(
Select *, 
    ROW_NUMBER() OVER (
    PARTITION BY ParcelID, 
                 PropertyAddress, 
                 SalePrice, 
                 SaleDate, 
                 LegalReference
                 ORDER BY 
                    UniqueID
    ) row_num
FROM nashville_housing
-- ORDER BY ParcelID
)
DELETE
From RowNumCTE
WHERE row_num > 1
-- Order By PropertyAddress

-------------------------------------------------------------------------------

-- Delete Unused Columns

ALTER TABLE nashville_housing
DROP COLUMN OwnerAddress, TaxDistrict, PropertyAddress

Select * 
From nashville_housing